## Combining data from multiple tables

While learning this concept we will use 2 different datasets
1. Air Quality Nitrate Data
2. Air Quality Particulate Matter Data

We will start by importing packages and then reading the two datasets

In [1]:
import pandas as pd

For non-standard datetime parsing, use ``to_datetime()`` after ``read_csv()`` instead of ``parse_dates``

In [3]:
air_quality_no2 = pd.read_csv(r"datasets\air_nitrate.csv",
                              parse_dates=True)

air_quality_no2 = air_quality_no2[["date.utc", "location",
                                   "parameter", "value"]]

air_quality_no2.head()

,date.utc,location,parameter,value
0,2019-06-21 00:00:00+00:00,FR04014,no2,20.0
1,2019-06-20 23:00:00+00:00,FR04014,no2,21.8
2,2019-06-20 22:00:00+00:00,FR04014,no2,26.5
3,2019-06-20 21:00:00+00:00,FR04014,no2,24.9
4,2019-06-20 20:00:00+00:00,FR04014,no2,21.4


In [4]:
air_quality_pm25 = pd.read_csv(r"datasets\air_particulate.csv",
                               parse_dates=True)

air_quality_pm25 = air_quality_pm25[["date.utc", "location",
                                     "parameter", "value"]]

air_quality_pm25.head()

,date.utc,location,parameter,value
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5


![combine](assets/combine.png)

I want to combine the measurements of $NO_2$ and $PM_25$, two tables with a similar structure, in a single table.

The ``concat()`` function performs concatenation operations of multiple tables along one of the axes (row-wise or column-wise).

In [5]:
air_quality = pd.concat([air_quality_pm25, air_quality_no2], axis=0)
air_quality.head()

,date.utc,location,parameter,value
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5


By default concatenation is along axis 0, so the resulting table combines the rows of the input tables. Let’s check the shape of the original and the concatenated tables to verify the operation:

In [6]:
print('Shape of the ``air_quality_pm25`` table: ', air_quality_pm25.shape)
print('Shape of the ``air_quality_no2`` table: ', air_quality_no2.shape)
print('Shape of the resulting ``air_quality`` table: ', air_quality.shape)

Shape of the ``air_quality_pm25`` table:  (1110, 4)
Shape of the ``air_quality_no2`` table:  (2068, 4)
Shape of the resulting ``air_quality`` table:  (3178, 4)


Hence, the resulting table has 3178 = 1110 + 2068 rows.

![merge](assets/merge.png)

For this example we will use two more datasets:
1. ``air_quality_stations.csv``
2. ``air_quality_parameters.csv``

Before proceeding lets load them in and see what it contains

In [7]:
stations_coord = pd.read_csv("datasets/air_q_stations.csv")
stations_coord.head()

,location,coordinates.latitude,coordinates.longitude
0,BELAL01,51.23619,4.38522
1,BELHB23,51.17030,4.34100
2,BELLD01,51.10998,5.00486
3,BELLD02,51.12038,5.02155
4,BELR833,51.32766,4.36226


Task: Add the station coordinates, provided by the stations metadata table, to the corresponding rows in the measurements table.

**Note:** The stations used in this example (FR04014, BETR801 and London Westminster) are just three entries enlisted in the metadata table. We only want to add the coordinates of these three to the measurements table, each on the corresponding rows of the air_quality table.

Using the ``merge()`` function, for each of the rows in the ``air_quality`` table, the corresponding coordinates are added from the ``air_quality_stations_coord`` table. Both tables have the column location in common which is used as a key to combine the information. By choosing the **left join**, only the locations available in the ``air_quality`` (left) table, i.e. ``FR04014``, ``BETR801`` and ``London Westminster``, end up in the resulting table. The merge function supports multiple join options similar to database-style operations.

In [8]:
air_quality = pd.merge(air_quality, stations_coord, how="left", on="location")
air_quality.head()

,date.utc,location,parameter,value,coordinates.latitude,coordinates.longitude
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0,51.20966,4.43182
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5,51.20966,4.43182
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5,51.20966,4.43182
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0,51.20966,4.43182
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5,51.20966,4.43182


Another example: Add the parameters’ full description and name, provided by the parameters metadata table, to the measurements table.

In [9]:
air_quality_parameters = pd.read_csv("datasets/air_q_parameters.csv")
air_quality_parameters.head()

,id,description,name
0,bc,Black Carbon,BC
1,co,Carbon Monoxide,CO
2,no2,Nitrogen Dioxide,NO2
3,o3,Ozone,O3
4,pm10,Particulate matter less than 10 micrometers in...,PM10


Compared to the previous example, there is no common column name. However, the ``parameter`` column in the ``air_quality`` table and the ``id`` column in the ``air_quality_parameters_name`` both provide the measured variable in a common format. The ``left_on`` and ``right_on`` arguments are used here (instead of just ``on``) to make the link between the two tables.

In [10]:
air_quality = pd.merge(air_quality, air_quality_parameters,
                       how='left', left_on='parameter', right_on='id')
air_quality.head()

,date.utc,location,parameter,value,coordinates.latitude,coordinates.longitude,id,description,name
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0,51.20966,4.43182,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5,51.20966,4.43182,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5,51.20966,4.43182,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0,51.20966,4.43182,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5,51.20966,4.43182,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5


Things to remember:
1. Multiple tables can be concatenated both column-wise and row-wise using the ``concat`` function.

2. For database-like merging/joining of tables, use the ``merge`` function.